In [1]:
# 导入工具包
import time
import os
from tqdm import tqdm

import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
%matplotlib inline

# 忽略烦人的红色提示
import warnings
warnings.filterwarnings("ignore")

# 获取计算硬件
# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device) 

device cuda:0


In [2]:
#图像预处理
from torchvision import transforms

# 训练集图像预处理：缩放裁剪、图像增强、转 Tensor、归一化
train_transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                     ])

# 测试集图像预处理-RCTN：缩放、裁剪、转 Tensor、归一化
test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize(
                                         mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])
                                    ])

In [3]:
# 数据集文件夹路径
dataset_dir = '../../data/lungImageSet_split'
train_path = os.path.join(dataset_dir, 'train')
test_path = os.path.join(dataset_dir, 'test')
print('训练集路径', train_path)
print('测试集路径', test_path)

from torchvision import datasets
# 载入训练集
train_dataset = datasets.ImageFolder(train_path, train_transform)
# 载入测试集
test_dataset = datasets.ImageFolder(test_path, test_transform)

print('训练集图像数量', len(train_dataset))
print('类别个数', len(train_dataset.classes))
print('各类别名称', train_dataset.classes)
print('测试集图像数量', len(test_dataset))
print('类别个数', len(test_dataset.classes))
print('各类别名称', test_dataset.classes)

# 各类别名称
class_names = train_dataset.classes
n_class = len(class_names)
# 映射关系：类别 到 索引号
train_dataset.class_to_idx
# 映射关系：索引号 到 类别
idx_to_labels = {y:x for x,y in train_dataset.class_to_idx.items()}

idx_to_labels

#定义数据加载器
from torch.utils.data import DataLoader

BATCH_SIZE = 64
epoch = 0

# 训练集的数据加载器
train_loader = DataLoader(train_dataset,
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          num_workers=4
                         )

# 测试集的数据加载器
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False,
                         num_workers=4
                        )

训练集路径 ../../data/lungImageSet_split/train
测试集路径 ../../data/lungImageSet_split/test
训练集图像数量 12000
类别个数 3
各类别名称 ['lung_aca', 'lung_n', 'lung_scc']
测试集图像数量 3000
类别个数 3
各类别名称 ['lung_aca', 'lung_n', 'lung_scc']


In [4]:
# 导入训练所需工具包
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

def evaluate_testset(model):
    # 在整个测试集上评估，返回分类评估指标日志
    
    # 交叉熵损失函数
    criterion = nn.CrossEntropyLoss() 
    loss_list = []
    labels_list = []
    preds_list = []
    
    with torch.no_grad():
        for images, labels in test_loader: # 生成一个 batch 的数据和标注
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images) # 输入模型，执行前向预测
            # outputs = outputs.logits
            # 获取整个测试集的标签类别和预测类别
            _, preds = torch.max(outputs, 1) # 获得当前 batch 所有图像的预测类别
            preds = preds.cpu().numpy()
            loss = criterion(outputs, labels) # 由 logit，计算当前 batch 中，每个样本的平均交叉熵损失函数值
            loss = loss.detach().cpu().numpy()
            outputs = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

            loss_list.append(loss)
            labels_list.extend(labels)
            preds_list.extend(preds)
        
    log_test = {}
    log_test['epoch'] = epoch
    # 计算分类评估指标
    log_test['test_loss'] = np.mean(loss)
    log_test['test_accuracy'] = accuracy_score(labels_list, preds_list)
    log_test['test_precision'] = precision_score(labels_list, preds_list, average='macro')
    log_test['test_recall'] = recall_score(labels_list, preds_list, average='macro')
    log_test['test_f1-score'] = f1_score(labels_list, preds_list, average='macro')
    
    return log_test

In [5]:
# 载入最佳模型作为当前模型
teacher_model = torch.load('./fine_tuned_pruned_resnet50-1.000.pth')
teacher_model.eval()
print(evaluate_testset(teacher_model))

{'epoch': 0, 'test_loss': 0.00052231346, 'test_accuracy': 1.0, 'test_precision': 1.0, 'test_recall': 1.0, 'test_f1-score': 1.0}


In [6]:
# 导入训练所需工具包
from torchvision import models
import torch.optim as optim
from torch.optim import lr_scheduler

student_model = models.resnet18(pretrained=False, num_classes=n_class)
student_model = student_model.to(device)

In [7]:
# 定义蒸馏损失函数
def distillation_loss(student_outputs, teacher_outputs, labels, temperature=1, alpha=0.5):
    hard_loss = F.cross_entropy(student_outputs, labels) * (1 - alpha)
    soft_loss = nn.KLDivLoss(reduction='batchmean')(F.log_softmax(student_outputs / temperature, dim=1),
                                                     F.softmax(teacher_outputs / temperature, dim=1)) * (alpha * temperature * temperature)
    return hard_loss + soft_loss

In [8]:
# 优化器和学习率调度器
optimizer = optim.Adam(student_model.parameters(), lr=0.01)
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# 训练轮次
EPOCHS = 80

# 学生模型训练
teacher_model.eval()
best_test_accuracy = 0
best_test_loss = 10000

In [9]:
# pip install wandb

In [10]:
import wandb

wandb.init(project='lung cancer', name=time.strftime('%m%d%H%M%S'))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jyjy2001lfx (jnjy). Use `wandb login --relogin` to force relogin


In [11]:
# 训练日志-测试集
df_test_log = pd.DataFrame()
log_test = {}
log_test['epoch'] = 0
log_test.update(evaluate_testset(student_model))
df_test_log = df_test_log.append(log_test, ignore_index=True)

In [14]:
for epoch in range(1, EPOCHS+1):
    
    print(f'Epoch {epoch}/{EPOCHS}')  # 打印当前训练轮数
    
    ## 训练阶段
    student_model.train()
    for images, labels in tqdm(train_loader):  # 获得一个 batch 的数据和标注
        images, labels = images.to(device), labels.to(device)

        # 前向传播教师模型
        with torch.no_grad():
            teacher_outputs = teacher_model(images)

        # 前向传播学生模型
        student_outputs = student_model(images)

        # 计算蒸馏损失
        loss = distillation_loss(student_outputs, teacher_outputs, labels, temperature=1, alpha=0.5)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    lr_scheduler.step()

    ## 测试阶段
    student_model.eval()
    log_test = evaluate_testset(student_model)
    df_test_log = df_test_log.append(log_test, ignore_index=True)
    wandb.log(log_test)
    print(log_test)
    
    if log_test['test_accuracy'] > best_test_accuracy: 
        # 删除旧的最佳模型文件(如有)
        old_best_checkpoint_path = './best_student_model.pth'.format(best_test_accuracy)
        if os.path.exists(old_best_checkpoint_path):
            os.remove(old_best_checkpoint_path)
        # 保存新的最佳模型文件
        new_best_checkpoint_path = './best_student_model.pth'.format(log_test['test_accuracy'])
        torch.save(student_model, new_best_checkpoint_path)
        print('保存新的最佳模型', './best_student_model.pth'.format(best_test_accuracy))
        best_test_accuracy = log_test['test_accuracy']
        best_test_loss = log_test['test_loss']
    elif log_test['test_accuracy'] == best_test_accuracy: 
        if log_test['test_loss'] < best_test_loss: 
            # 删除旧的最佳模型文件(如有)
            old_best_checkpoint_path = './best_student_model.pth'.format(best_test_accuracy)
            if os.path.exists(old_best_checkpoint_path):
                os.remove(old_best_checkpoint_path)
            # 保存新的最佳模型文件
            new_best_checkpoint_path = './best_student_model.pth'.format(log_test['test_accuracy'])
            torch.save(student_model, new_best_checkpoint_path)
            print('保存新的最佳模型', './best_student_model.pth'.format(best_test_accuracy))
            best_test_accuracy = log_test['test_accuracy']
            best_test_loss = log_test['test_loss']
        
# 保存学生模型
torch.save(student_model, 'student_model.pth')

Epoch 1/80


100%|██████████| 188/188 [00:33<00:00,  5.59it/s]


{'epoch': 1, 'test_loss': 1.094175, 'test_accuracy': 0.8683333333333333, 'test_precision': 0.8969591474333042, 'test_recall': 0.8683333333333333, 'test_f1-score': 0.8679982632908311}
保存新的最佳模型 ./best_student_model.pth
Epoch 2/80


100%|██████████| 188/188 [00:33<00:00,  5.63it/s]


{'epoch': 2, 'test_loss': 0.6175906, 'test_accuracy': 0.9236666666666666, 'test_precision': 0.9260603026164338, 'test_recall': 0.9236666666666666, 'test_f1-score': 0.9243323859515981}
保存新的最佳模型 ./best_student_model.pth
Epoch 3/80


...

{'epoch': 78, 'test_loss': 0.00828541, 'test_accuracy': 1.0, 'test_precision': 1.0, 'test_recall': 1.0, 'test_f1-score': 1.0}
Epoch 79/80


100%|██████████| 188/188 [00:34<00:00,  5.47it/s]


{'epoch': 79, 'test_loss': 0.008108442, 'test_accuracy': 1.0, 'test_precision': 1.0, 'test_recall': 1.0, 'test_f1-score': 1.0}
Epoch 80/80


100%|██████████| 188/188 [00:34<00:00,  5.51it/s]


{'epoch': 80, 'test_loss': 0.0075982264, 'test_accuracy': 1.0, 'test_precision': 1.0, 'test_recall': 1.0, 'test_f1-score': 1.0}


In [15]:
print(evaluate_testset(student_model))

{'epoch': 80, 'test_loss': 0.0075982264, 'test_accuracy': 1.0, 'test_precision': 1.0, 'test_recall': 1.0, 'test_f1-score': 1.0}


In [16]:
print(evaluate_testset(teacher_model))

{'epoch': 80, 'test_loss': 0.00052231346, 'test_accuracy': 1.0, 'test_precision': 1.0, 'test_recall': 1.0, 'test_f1-score': 1.0}


In [17]:
# 载入最佳模型作为当前模型
best_student_model = torch.load('./best_student_model.pth')
best_student_model.eval()
print(evaluate_testset(best_student_model))

{'epoch': 80, 'test_loss': 0.004158932, 'test_accuracy': 1.0, 'test_precision': 1.0, 'test_recall': 1.0, 'test_f1-score': 1.0}
